In [1]:
import sys 
import os
sys.path.append(r'/Users/kostastsampourakis/Desktop/code/Python/projects/neuralssm/src/neuralssm')
import argparse
import jax.numpy as jnp
import jax.random as jr
import time
import util.io
import experiment_descriptor as ed
import misc
import util.plot
import matplotlib.pyplot as plt
import tensorflow_probability.substrates.jax.distributions as tfd # type: ignore
import main 
exp_root = '/Users/kostastsampourakis/Desktop/code/Python/projects/neuralssm/src/neuralssm/exps/'
data_root = '/Users/kostastsampourakis/Desktop/code/Python/projects/neuralssm/src/neuralssm/data/experiments/'

## Run experiment

In [2]:
run = False
if run:
    exp_file = exp_root + 'exp0.txt'
    sys.argv = ['', 'trials', '13434', '1', '2', exp_file] 
    args = main.parse_args()
    args.func(args)

## View experiment

In [3]:
view = False
trial = 1
if view:
    for trial in range(1, 3):
        exp_file = exp_root + 'exp0.txt'
        sys.argv = ['', 'view', '-t', f'{trial}', exp_file]
        args = main.parse_args()
        args.func(args)

In [4]:
view = False
trial = 1
if view:
    for trial in range(1, 2):
        exp_file = exp_root + 'new_exp.txt'
        sys.argv = ['', 'view', '-t', f'{trial}', exp_file]
        args = main.parse_args()
        args.func(args)

In [5]:
view = False
trial = 1
if view:
    for trial in range(1, 2):
        exp_file = exp_root + 'lgssm_bpf_mcmc.txt'
        sys.argv = ['', 'view', '-t', f'{trial}', exp_file]
        args = main.parse_args()
        args.func(args)

In [6]:
view = False
start_trial = 1
end_trial = 11
if view:
    for trial in range(1, 5): #range(start_trial, end_trial):
        exp_file = exp_root + 'lgssm_tsnl_test.txt'
        sys.argv = ['', 'view', '-t', f'{trial}', '-o', 'true', exp_file]
        args = main.parse_args()
        args.func(args)

In [7]:
view = False
if view:
    exp_root = '/Users/kostastsampourakis/Desktop/code/Python/projects/neuralssm/src/neuralssm/exps/'
    for trial in range(1, 11):
        exp_file = exp_root + 'lgssm_abc.txt'
        sys.argv = ['', 'view', '-t', f'{trial}', exp_file]
        args = main.parse_args()
        args.func(args)

In [8]:

def get_error(exp_dir):

    errors = []
    mmds = []
    true_cps_trials = []
    observations_trials = []
    all_emissions_trials = []
    post_cps_trials = []
    
    for trial in range(start_trial, ntrials+1):
        

        try:
            # print(os.path.join(sim_dir + inf_dir, 'gt'))
            # # error, _ = util.io.load(os.path.join(sim_dir + inf_dir, 'error'))
            all_emissions = util.io.load(os.path.join(exp_dir + f'/{trial}', 'all_emissions'))
            (true_ps, true_cps), observations =  util.io.load(exp_dir + f'/{trial}' + '/gt')
            # mmd = util.io.load(sim_dir + inf_dir + '/mmd')
            # mmds.append(mmd)
            # info = util.io.load(os.path.join(sim_dir+inf_dir, 'info'))
            _, post_cps = util.io.load(os.path.join(exp_dir + f'/{trial}', 'posterior'))

            # # errors.append(error)
            all_emissions_trials.append(all_emissions)
            post_cps_trials.append(post_cps)
            true_cps_trials.append(true_cps)
            # observations_trials.append(observations)

        except FileNotFoundError:

            print(f'File not found')

            continue

    out = {
        'errors': jnp.array(errors),
        'mmd' : jnp.array(mmds),
        'true_cps_trials': true_cps_trials,
        'observations_trials': observations_trials,
        'all_emissions_trials': jnp.array(all_emissions_trials),
        'post_cps_trials': jnp.array(post_cps_trials)
        }

    return out

def get_exp_dir(inf,
                sim,
                state_dim,
                emission_dim,
                num_timesteps,
                vars,
                num_samples,
                num_rounds,
                train_on,
                subsample,
                lag,
                dhidden):

    if inf == 'snl':

        inf_dir = f'nde/snl/samples_{num_samples}_rounds_{num_rounds}_train_on_{train_on}_mcmc_steps_1000/maf_nmades_5_dhidden_32_nhiddens_5'

    elif inf == 'tsnl':

        inf_dir = f'nde/tsnl/samples_{num_samples}_rounds_{num_rounds}_lag_{lag}_subsample_{subsample}_train_on_{train_on}_mcmc_steps_1000/maf_nmades_5_dhidden_{dhidden}_nhiddens_5'

    if sim == 'lvssm':

        sim_dir = data_root + f'{sim}/emission-dim_{emission_dim}_num-timesteps_{num_timesteps}_target-vars_{vars}/'
        
    elif sim == 'svssm':

        sim_dir = data_root + f'{sim}/state-dim_{state_dim}_emission-dim_{emission_dim}_num-timesteps_{num_timesteps}_target-vars_{vars}/'

    elif sim == 'lgssm':

        sim_dir = data_root + f'{sim}/state-dim_{state_dim}_emission-dim_{emission_dim}_num-timesteps_{num_timesteps}_target-vars_{vars}/'

    exp_dir = sim_dir + inf_dir

    return exp_dir
    
view = True

if view:

    state_dim = 1
    emission_dim = 1
    dhidden = 32
    vars = 'd4'
    start_trial = 1
    ntrials = 5

    num_timesteps = 100
    num_samples = 100
    num_rounds = 5
    train_on = 'best'
    subsample = 1.0
    lag = 2

    snl_exp_dir = get_exp_dir('snl', 'svssm', state_dim, emission_dim, num_timesteps, vars, num_samples, num_rounds, train_on, subsample, lag, dhidden)
    tsnl_exp_dir = get_exp_dir('tsnl', 'svssm', state_dim, emission_dim, num_timesteps, vars, num_samples, num_rounds, 'all', subsample, lag, dhidden)

    out_snl = get_error(snl_exp_dir)
    out_tsnl = get_error(tsnl_exp_dir)

    post_trials_snl = out_snl['post_cps_trials']
    true_cps_snl = out_snl['true_cps_trials']
    post_trials_tsnl = out_tsnl['post_cps_trials']
    true_cps_tsnl = out_tsnl['true_cps_trials']

In [ ]:
import os
import signal
import psutil
import socket

# Example: Closing an open socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(("localhost", 6001))

# Forcefully close the socket
# sock.close()

# for conn in psutil.net_connections(kind='tcp'):
#     try:
#         sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
#         sock.bind((conn.laddr.ip, conn.laddr.port))
#         sock.close()
#     except Exception:
#         pass

# # Find the process holding the port (e.g., 5000)
# port = 5000
# for conn in psutil.net_connections(kind='tcp'):
#     if conn.laddr.port == port:
#         os.kill(conn.pid, signal.SIGTERM)  # SIGKILL for forceful shutdown


<socket.socket fd=101, family=2, type=1, proto=0, laddr=('127.0.0.1', 6001)>

In [114]:
sock.listen(5)  # Allow up to 5 pending connections

print("Waiting for connections...")

while True:
    conn, addr = sock.accept()  # Accept an incoming connection
    print(f"Connection from {addr}")  # addr contains (IP, port)
    
    data = conn.recv(1024)  # Receive up to 1024 bytes from the client
    print(f"Received: {data.decode()}")
    
    conn.sendall(b"Hello, client!")  # Send a response
    conn.close()  # Close the connection

Waiting for connections...


KeyboardInterrupt: 

In [118]:
import os

pid = os.getpid()  # Get current process ID
print(f"Current PID: {pid}")
psutil.net_connections(kind='tcp')
# for conn in 
#     print(conn)
#     if conn.pid == pid:
#         print(f"Active connection: {conn.laddr} <--> {conn.raddr}")

Current PID: 26721


AccessDenied: (pid=48826)

In [84]:
import numpy as np
from jax import random as jr, numpy as jnp
import matplotlib.pyplot as plt
from scipy.stats import norm
import time
key = jr.PRNGKey(int(time.time()))

# Define Gaussian mixture for p and q
def gaussian_mixture_pdf(theta, means, weights, sigma=1):
    """Compute the PDF of a Gaussian mixture at given points."""
    pdf = np.zeros_like(theta)
    for mean, weight in zip(means, weights):
        pdf += weight * norm.pdf(theta, loc=mean, scale=sigma)
    return pdf

# Define parameters for q (mixture of Gaussians)
num_q = 2
key, subkey = jr.split(key)
means_q = 1 * jr.normal(subkey, (num_q,))  # Random means
weights_q = jnp.ones(num_q)/num_q

# Define parameters for p (another mixture of Gaussians)
key, subkey = jr.split(key)
num_p = 2
means_p = 1 * jr.normal(subkey, (num_p,))  # Random means
weights_p = jnp.ones(num_p)/num_p

# Compute log density ratio
theta_vals = np.linspace(-5, 5, 300)
p_vals = gaussian_mixture_pdf(theta_vals, means_p, weights_p)
q_vals = gaussian_mixture_pdf(theta_vals, means_q, weights_q)
log_r_vals = np.log(p_vals / q_vals)

# Plot the log density ratio
plt.figure(figsize=(8, 5))
plt.plot(theta_vals, log_r_vals, label=r"$\log r(\theta) = \log \frac{p(\theta)}{q(\theta)}$", color="red")
for mean in means_q:
    plt.axvline(mean, color="blue", linestyle="--", label="Means of $q$")
for mean in means_p:    
    plt.axvline(mean, color="black", linestyle="--", label="Means of $p$")
plt.xlabel(r"$\theta$")
plt.ylabel(r"$\log r(\theta)$")
plt.title("Log Density Ratio for 1D Gaussian Mixtures")
plt.legend()
plt.grid(True)
plt.show()

In [9]:
fig1, ax1 = plt.subplots(1, 5, figsize=(5, 1))

for i in range(0, 5):

    try:

        ax1[i].hist(post_trials_snl[i].flatten(), bins=100, density=True)
        ax1[i].axvline(x=true_cps_snl[i], color='r')

    except:

        print(f'No data for trial {i}')

fig2, ax2 = plt.subplots(1, 5, figsize=(5, 1))

for i in range(0, 5):

    try:

        ax2[i].hist(post_trials_tsnl[i].flatten(), bins=100, density=True)
        ax2[i].axvline(x=true_cps_tsnl[i], color='r')

    except:

        print(f'No data for trial {i}')

plt.show()

In [11]:
print(true_cps_tsnl)
print(post_trials_tsnl[:, 0])

[Array([0.03491543], dtype=float32), Array([-0.14728509], dtype=float32), Array([-0.06451591], dtype=float32), Array([0.00428553], dtype=float32), Array([0.04077392], dtype=float32)]
[[-0.11280572]
 [ 0.01050321]
 [ 0.10671747]
 [ 0.02230529]
 [ 0.05511151]]


In [12]:
from util.misc import kde_error
import jax.scipy.stats as jss

for trial in range(ntrials):

    try:
        
        kernel_points = all_post_trials[trial].T
        kde = jss.gaussian_kde(kernel_points)
        kderr = -jnp.log(kde.evaluate(true_cps_trials[trial]))
        print('kde error =', kderr)
        print('RMSE =', jnp.linalg.norm(jnp.mean(all_post_trials[trial], axis=0)-true_cps_trials[trial]))

    except IndexError:

        continue

NameError: name 'all_post_trials' is not defined

In [ ]:
from util.plot import plot_hist_marginals


for t in range(ntrials):

    try:

        plot_hist_marginals(all_post_trials[t], gt = true_cps_trials[t])

    except IndexError:

        print('IndexError')

        continue

    plt.show()


IndexError
IndexError
IndexError
IndexError
IndexError


In [ ]:
n_samples = int(all_emissions_trials.shape[1]/5)
all_emissions=all_emissions_trials.reshape(ntrials, 5, n_samples, 30, 1)

for trial in range(ntrials):

    fig, ax = plt.subplots(1, 5, figsize=(5, 1))
    
    for r in range(5):

        emissions = all_emissions[trial, r]

        for i in range(10):

            ax[r].plot(emissions[i], label=f'{i}')

    plt.show()
    plt.close()

TypeError: cannot reshape array of shape (5, 50, 100, 1) (size 25000) into shape (5, 5, 10, 30, 1) (size 7500)